In [1]:
import pandas as pd
import time, os, shutil
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np  
from sklearn.cluster import KMeans  
from sklearn.cluster import DBSCAN
from nltk.parse import CoreNLPParser

pos_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='pos')

/home/betty35/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0412 20:19:48.297299 140570116118272 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [5]:
data_source = 'output/sub-set/'
file_name = 'Fire HD'+'.csv'
df = pd.read_csv(data_source+file_name, sep="\t", error_bad_lines=False)
reviews = pd.DataFrame(df['text'])

In [12]:
reviews['POS'] = reviews.text.apply(lambda x: pos_tagger.tag(x.split()))

In [14]:
reviews.to_csv(r'/mnt/doc/School/USYD/Capstone/workspace/data_testing/output/sub-set/fireHD-tag.csv',sep='\t')

In [16]:
reviews['POS'][0:10].tolist()

[[('I', 'PRP'),
  ('find', 'VBP'),
  ('that', 'DT'),
  ('is', 'VBZ'),
  ('not', 'RB'),
  ('as', 'RB'),
  ('easy', 'JJ'),
  ('to', 'TO'),
  ('use', 'VB'),
  ('as', 'IN'),
  ('my', 'PRP$'),
  ('regular', 'JJ'),
  ('Kindle', 'NNP'),
  ('Fire', 'NNP')],
 [('When', 'WRB'),
  ('I', 'PRP'),
  ('am', 'VBP'),
  ('reading', 'VBG'),
  ('the', 'DT'),
  ('kindle', 'NN'),
  ('it', 'PRP'),
  ('freezes', 'VBZ'),
  ('up', 'RP'),
  ('-LRB-', '-LRB-'),
  ('because', 'IN'),
  ('you', 'PRP'),
  ('hold', 'VBP'),
  ('it', 'PRP'),
  ('on', 'IN'),
  ('the', 'DT'),
  ('side', 'NN'),
  ('-RRB-', '-RRB-')],
 [('Also', 'RB'),
  ('you', 'PRP'),
  ('have', 'VBP'),
  ('to', 'TO'),
  ('get', 'VB'),
  ('out', 'IN'),
  ('of', 'IN'),
  ('it', 'PRP'),
  ('to', 'TO'),
  ('brighten', 'VB'),
  ('or', 'CC'),
  ('darken', 'VB'),
  ('your', 'PRP$'),
  ('text', 'NN'),
  ('that', 'IN'),
  ('you', 'PRP'),
  ('are', 'VBP'),
  ('reading', 'VBG')],
 [('Lastly', 'RB'),
  (',', ','),
  ('the', 'DT'),
  ('advertisements', 'NNS'),
  ('th

In [17]:
def getMainWords(pos):
    result = '';
    for x in pos:
        if x[1][0:2] in ['NN','VB']:
            result= result+x[0]+' '
    if len(result)>0:
        result= result[0:len(result)-1]
    return result

reviews['main'] = list(map(getMainWords,reviews['POS']))
reviews = reviews.loc[reviews['main'].apply(len)>0,['text','main']] #filtering out sentences without any nouns or verbs

reviews = reviews.reset_index(drop=True)# reset index

In [2]:
# If load data with 'text' and 'main'
data_source = 'output/sub-set/'
file_name = 'fireHD-tag'+'.csv'
reviews = pd.read_csv(data_source+file_name, sep="\t", error_bad_lines=False).loc[:,['text','main']]

In [3]:
# Load google Universal Sentence Encoder
module_dir ="downloads/encoder-DNA" #"downloads/encoder"
embed = hub.Module(module_dir)

Instructions for updating:
Colocations handled automatically by placer.


W0412 20:21:22.740859 140570116118272 deprecation.py:323] From /home/betty35/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    start_time=time.time()
    reviews['sentence_embedding'] = pd.Series(list(session.run(embed(list(reviews['text'])))))
    end_time1=time.time()
    reviews['words_embedding'] = pd.Series(list(session.run(embed(list(reviews['main'])))))
    end_time2=time.time()
    print('time1:',(end_time1-start_time),' time2:',(end_time2-end_time1))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0412 20:21:53.503647 140570116118272 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0412 20:21:58.211255 140570116118272 saver.py:1483] Saver not created because there are no variables in the graph to restore


time1: 5.509376764297485  time2: 3.5216400623321533


In [24]:
#sub=reviews.loc[:,['text','main']]
#sub.to_csv(r'/mnt/doc/School/USYD/Capstone/workspace/data_testing/output/sub-set/fireHD-tag.csv',sep='\t')

In [5]:
# Preparing datasets
X1 = np.array(reviews['sentence_embedding'].tolist())
X2 = np.array(reviews['words_embedding'].tolist())

In [26]:
# Clustering
time1 = time.time()
db_s = DBSCAN(eps=0.4).fit(X1)
print('time for clustering sentence embeddings:', (time.time()-time1))
time1 = time.time()
db_w = DBSCAN(eps=0.4).fit(X2)
print('time for clustering words embeddings:',(time.time()-time1))
# Get labels
labels_s = db_s.labels_.tolist()
labels_w = db_w.labels_.tolist()
reviews['label_db_sentence'] = labels_s
reviews['label_db_word'] = labels_w
n_sentence= len(set(labels_s)) - (1 if -1 in labels_s else 0)
n_words=len(set(labels_w)) - (1 if -1 in labels_w else 0)

time for clustering sentence embeddings: 10.899672746658325
time for clustering words embeddings: 10.898835897445679


In [27]:
print('total sentences:',len(list(labels_s)))
print('clusters for sentences:',n_sentence, 'noise:',list(labels_s).count(-1))
print('clusters for words:',n_words, 'noise:',list(labels_w).count(-1))

total sentences: 3355
clusters for sentences: 2 noise: 854
clusters for words: 15 noise: 1041


In [19]:
def save_results_dir(product_name, method, params):
    dir_pre = '/home/betty35/桌面/Capstone/workspace/data_testing/output/'
    dir_ = dir_pre+product_name+'/'+method+'/'+params+'/'
    return dir_


def make_dir(dir_):
    if not os.path.exists(dir_):
        try:
            os.makedirs(dir_)
        except FileExistsError:
            pass

def clean_dir(dir_):
    for file_ in os.listdir(dir_):
        file_dir = os.path.join(dir_, file_)
        try:
            if os.path.isfile(file_dir):
                os.unlink(file_dir)
            elif os.path.isdir(file_dir): shutil.rmtree(file_dir)
        except Exception as e:
            print(e)

def make_or_clean_dir(dir_):
    make_dir(dir_)
    clean_dir(dir_)

In [28]:
product_title='Fire HD'
params='0.8'
dir1 = save_results_dir(product_title,'dbscan',params+'/sentence')
make_or_clean_dir(dir1)
for i in range(0,n_sentence):    
    temp = reviews.loc[reviews['label_db_sentence']==i,['text']]
    file_name = str(i)+'.txt'
    np.savetxt(r''+dir1+file_name, temp.values, fmt='%s')

noise_s = temp = reviews.loc[reviews['label_db_sentence']==-1,['text']] 
np.savetxt(r''+dir1+'noise.txt', temp.values, fmt='%s')

dir1 = save_results_dir(product_title,'dbscan',params+'/word')
make_or_clean_dir(dir1)
for i in range(0,n_words):
    temp = reviews.loc[reviews['label_db_word']==i,['text']]
    file_name = str(i)+'.txt'
    np.savetxt(r''+dir1+file_name, temp.values, fmt='%s')
noise_s = temp = reviews.loc[reviews['label_db_word']==-1,['text']] 
np.savetxt(r''+dir1+'noise.txt', temp.values, fmt='%s')